In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta
import mplfinance as mpf
import os
from sklearn.preprocessing import MinMaxScaler

# Refresh Datasets refresh_datasets()

In [2]:
def refresh_datasets(start_date='2006-04-11'):

#DOWNLOAD DATA

    data = yf.download(["^GSPC", '^VIX', '^IRX', '^TNX', 'HYG', 'XLE', 'XLU', 'XLY'], start = start_date, group_by='ticker')

    #unpacking "data"
    sp500 = data[('^GSPC')]
    vix = data[('^VIX')]
    us2yryld = data[("^IRX")]
    us10yryld = data[("^TNX")]
    hyg = data[("HYG")]
    energysp500 = data[("XLE")]
    utilitysp500 = data[("XLU")]
    consumersp500 = data[("XLY")]

    usunemprate = pd.read_csv("USUNRATE.csv") ##https://fred.stlouisfed.org/series/UNRATE

#DATA CLEANUP
    vix = vix.drop(["Volume", 'Open', 'High', 'Low', 'Adj Close'], axis=1)
    sp500 = sp500.drop(["Adj Close"], axis=1)
    us2yryld = us2yryld.drop(["Volume", 'Open', 'High','Low', 'Adj Close'], axis=1)
    us10yryld = us10yryld.drop(["Volume", 'Open', 'High','Low', 'Adj Close'], axis=1)
    hyg = hyg.drop(["Volume", 'Open', 'High','Low', 'Adj Close'], axis=1)
    energysp500 = energysp500.drop(["Volume", 'Open', 'High','Low', 'Adj Close'], axis=1)
    utilitysp500 = utilitysp500.drop(["Volume", 'Open', 'High','Low', 'Adj Close'], axis=1)
    consumersp500 = consumersp500.drop(["Volume", 'Open', 'High','Low', 'Adj Close'], axis=1)

#RENAMING COLUMNS
    vix.columns = ['vix_close']
    us2yryld.columns = ['us2yryld_close']
    us10yryld.columns = ['us10yryld_close']
    usunemprate = usunemprate.rename(columns={'DATE': 'Date', 'UNRATE': 'unemployment_rate'})
    hyg.columns = ['hyg_close']
    energysp500.columns = ['energysp500_close']
    utilitysp500.columns = ['utilitysp500_close']
    consumersp500.columns = ['consumersp500_close']

#YIELDS STRUCTURING
    yield_ratio = pd.concat([us10yryld, us2yryld], axis=1)
    yield_ratio['yield_ratio'] = yield_ratio['us2yryld_close']/yield_ratio['us10yryld_close']
        
    return sp500, vix, yield_ratio, hyg, energysp500, utilitysp500, consumersp500, usunemprate
        

# Creating Technical Analysis Columns

In [5]:
def create_graphs(sp500, vix, yield_ratio, usunemprate,
              rsi = 20,
              sma_st=50,
              sma_mt=100,
              sma_lt=200,
              bbands_length=20,
              bbands_std=2.0,
              recent_days_graphed = 500,
              rsi_low = 35,
              rsi_high = 65):
    
    def ta_created():
        #SP500
            #RSI
        sp500_ta = sp500
        sp500_ta[f"sp500_rsi_{rsi}"] = ta.rsi(close = sp500_ta.Close, length=rsi)

            #SMA
        sp500_ta[f"sp500_SMA_{sma_lt}"] = ta.sma(sp500_ta['Close'], sma_lt)
        sp500_ta[f"sp500_SMA_{sma_mt}"] = ta.sma(sp500_ta['Close'], sma_mt)
        sp500_ta[f"sp500_SMA_{sma_st}"] = ta.sma(sp500_ta['Close'], sma_st)

            #Bollinger Bands
        my_bbands = ta.bbands(close = sp500_ta.Close, length=bbands_length, std=bbands_std)
        sp500_ta=sp500_ta.join(my_bbands)


        #VIX, 2yr/10 Yr Yield and US unempl rate scaled
        scaler = MinMaxScaler(feature_range=(-1,1))
        
        vix_ta = vix
        vix['scaled_vix'] = scaler.fit_transform(vix['vix_close'].values.reshape(-1,1))
        
        yield_ratio_ta = yield_ratio
        yield_ratio['scaled_yield_ratio'] = scaler.fit_transform(yield_ratio['yield_ratio'].values.reshape(-1,1))
        
        usunemprate_ta = usunemprate
        usunemprate['scaled_unemployment_rate'] = scaler.fit_transform(usunemprate['unemployment_rate'].values.reshape(-1,1))

        return sp500_ta, vix_ta, yield_ratio_ta, usunemprate_ta

    def graphing_data():
        
        sp500_ta, vix_ta, yield_ratio_ta, usunemprate_ta = ta_created()
        sp500_graph = sp500_ta[-recent_days_graphed:]
        vix_graph = vix_ta[-recent_days_graphed:]
        yield_ratio_graph = yield_ratio_ta[-recent_days_graphed:]
        usunemprate_graph = usunemprate_ta[-recent_days_graphed:]
        
        signal = sp500_ta[-1::]
        def today_signal():
            sma_st_GT_sma_mt = signal.iloc[0][f'sp500_SMA_{sma_st}'] > signal.iloc[0][f"sp500_SMA_{sma_mt}"]
            sma_mt_GT_sma_lt = signal.iloc[0][f'sp500_SMA_{sma_mt}'] > signal.iloc[0][f"sp500_SMA_{sma_lt}"]
            rsi_LT_rsiLOW = signal.iloc[0][f'sp500_rsi_{rsi}'] < rsi_low
            rsi_GT_rsiHIGH = signal.iloc[0][f'sp500_rsi_{rsi}'] > rsi_high
            
            yield_ratio_signal = yield_ratio_ta[-1::]
            
            return sma_st_GT_sma_mt, sma_mt_GT_sma_lt, rsi_LT_rsiLOW, rsi_GT_rsiHIGH


        sma_st_GT_sma_mt, sma_mt_GT_sma_lt, rsi_LT_rsiLOW, rsi_GT_rsiHIGH = today_signal()
        st_mt_condition = (sp500_graph[f'sp500_SMA_{sma_st}'] > sp500_graph[f'sp500_SMA_{sma_mt}']) == sma_st_GT_sma_mt
        mt_lt_condition = (sp500_graph[f'sp500_SMA_{sma_mt}'] > sp500_graph[f'sp500_SMA_{sma_lt}']) == sma_mt_GT_sma_lt  
        rsi_low_condition = (sp500_graph[f'sp500_rsi_{rsi}'] < rsi_low) == rsi_LT_rsiLOW
        rsi_high_condition = (sp500_graph[f'sp500_rsi_{rsi}'] > rsi_low) == rsi_GT_rsiHIGH

        sp500_signal = sp500_graph.loc[
                                        (sp500_graph[f'sp500_SMA_{sma_st}'] > sp500_graph[f'sp500_SMA_{sma_mt}']) == sma_st_GT_sma_mt & (sp500_graph[f'sp500_SMA_{sma_mt}'] > sp500_graph[f'sp500_SMA_{sma_lt}']) == sma_mt_GT_sma_lt & (sp500_graph[f'sp500_rsi_{rsi}'] < rsi_low) == rsi_LT_rsiLOW & 
                                        (sp500_graph[f'sp500_rsi_{rsi}'] > rsi_low) == rsi_GT_rsiHIGH
                                        ]

        # apds = [
            
        # mpf.make_addplot(sp500_graph[f'BBL_{bbands_length}_{bbands_std}'], color = 'green', panel=0),
        # mpf.make_addplot(sp500_graph[f'BBU_{bbands_length}_{bbands_std}'], color = 'red', panel=0),

    # signal graph
        # mpf.make_addplot(sp500_signal['Close'],
                        #  type='scatter',
                        #  marker = '^',
                        #  markersize = 100,
                        #  panel = 0)

    #SMA Graph
    #     mpf.make_addplot(sp500_graph[f'sp500_SMA_{sma_lt}'],
    #                      panel=2,
    #                      alpha=1,
    #                      secondary_y = False),
    #     mpf.make_addplot(sp500_graph[f'sp500_SMA_{sma_mt}'],
    #                      panel=2,
    #                      alpha=1,
    #                      secondary_y = False),
    #     mpf.make_addplot(sp500_graph[f'sp500_SMA_{sma_st}'],
    #                      panel=2,
    #                      alpha=1,
    #                      secondary_y = False),

    # #RSI `Graph`
    #     mpf.make_addplot(sp500_graph[f'sp500_rsi_{rsi}'],
    #                      type='bar',
    #                      color='blue',
    #                      panel=2,
    #                      alpha = 0.5,
    #                      secondary_y = True,
    #                      ylabel = f'RSI_{rsi}'),

    # #VIX
    #     mpf.make_addplot(vix_graph['vix_close'],
    #                      color='black',
    #                      panel=1,
    #                      alpha = 1,
    #                      ylabel = "VIX"),

    # #US Yields
    #     mpf.make_addplot(yield_ratio_graph['scaled_yield_ratio'],
    #                      color='blue',
    #                      panel=3,
    #                      alpha = 1,
    #                      ylabel = "2yr/10yr"),
    
    # #Unemployment Rate
    #     mpf.make_addplot(usunemprate_graph['unemployment_rate'],
    #                      panel = 4,
    #                      ylabel = "Unempl.")
        
        # ]
        
    
        # return mpf.plot(sp500_graph,
        #                 style='classic',
        #                 figscale = 1.5,
        #                 show_nontrading=True, 
        #                 addplot=apds,
        #                 # panel_ratios =(3,2,2,2,2),
        #                 tight_layout = False)
        
        return sp500_signal
    return graphing_data()
    

## Running Code

In [4]:
#Refresh datasets to date (default start date 2006-04-01)
sp500, vix, yield_ratio, hyg, energysp500, utilitysp500, consumersp500, usunemprate = refresh_datasets()

[*********************100%***********************]  8 of 8 completed


In [6]:
create_graphs(sp500, vix, yield_ratio, usunemprate, recent_days_graphed=1000, sma_lt = 250, sma_mt = 55, sma_st=22)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
signal = sp500[-1::]
sp500_graph = sp500
sma_st_GT_sma_mt = signal.iloc[0]['sp500_SMA_22'] > signal.iloc[0]['sp500_SMA_55']
sma_mt_GT_sma_lt = signal.iloc[0]['sp500_SMA_55'] > signal.iloc[0]['sp500_SMA_250']

# sp500_filtered = sp500_graph[(sp500_graph['sp500_SMA_22'] > sp500_graph['sp500_SMA_55']) == sma_st_GT_sma_mt
#                             & (sp500_graph['sp500_SMA_55'] > sp500_graph['sp500_SMA_250']) == sma_mt_GT_sma_lt
#                             ]

st_mt_condition = (sp500_graph['sp500_SMA_22'] > sp500_graph['sp500_SMA_55']) == sma_st_GT_sma_mt
mt_lt_condition = (sp500_graph['sp500_SMA_55'] > sp500_graph['sp500_SMA_250']) == sma_mt_GT_sma_lt

sp500_filtered = sp500_graph.loc[st_mt_condition & mt_lt_condition]
sp500_filtered
# mpf.plot(sp500_filtered)
# sp500_graph.head(150)

# signal[0]['sp500_SMA_22'] > signal[0]['sp500_SMA_55']
# sma_mt_GT_sma_lt = signal['sp500_SMA_55'] > signal['sp500_SMA_250']

# sma_mt_GT_sma_lt
# sma_mt_GT_sma_lt = signal[f'sp500_SMA_{sma_mt}'] > signal[f"sp500_SMA_{sma_lt}"]
# rsi_LT_rsiLOW = signal[f'sp500_rsi_{rsi}'] < rsi_low
# rsi_GT_rsiHIGH = signal[f'sp500_rsi_{rsi}'] > rsi_high

# yield_ratio_signal = yield_ratio_ta[-1::]

# sma_st_GT_sma_mt, sma_mt_GT_sma_lt, rsi_LT_rsiLOW, rsi_GT_rsiHIGH = today_signal()

# sp500_graph = sp500_ta[-recent_days_graphed:]
# vix_graph = vix_ta[-recent_days_graphed:]
# yield_ratio_graph = yield_ratio_ta[-recent_days_graphed:]
# usunemprate_graph = usunemprate_ta[-recent_days_graphed:]

# sp500_signal = sp500_graph[
#     (sp500_graph[f'sp500_SMA_{sma_st}'] > sp500_graph[f'sp500_SMA_{sma_mt}']).reset_index(drop=True) == (signal[f'sp500_SMA_{sma_st}'] > signal[f"sp500_SMA_{sma_mt}"]).reset_index(drop=True)
#     # (sp500_graph[f'sp500_SMA_{sma_mt}'] > sp500_graph[f'sp500_SMA_{sma_lt}']) ==  sma_mt_GT_sma_lt &
#     # (sp500_graph[f'sp500_rsi_{rsi}'] < rsi_low) == rsi_LT_rsiLOW &
#     # (sp500_graph[f'sp500_rsi_{rsi}'] > rsi_high) == rsi_GT_rsiHIGH
#     ]
